Data from: http://zhao-nas.bio.cmu.edu:5000/fsdownload/aBDx29J7H/Ensemble%20learning%20data_shared

In [3]:
#!pip install torch-summary
from pathlib import Path
import cv2 , os, numpy as np, torch, pandas as pd, tqdm as tqdm, PIL.Image as Image
from pylab import rcParams
import datetime
# 
from torch import nn, optim
import torch.nn.functional as F
import torchvision.transforms as T
# from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from torchsummary import summary
from pytorch_lightning.loops.fit_loop import FitLoop
from pytorch_lightning.loops import Loop
#

from numpy.lib.function_base import select
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#
#import efficientnet_pytorch
import pytorch_lightning as pl, torchmetrics

In [10]:
class PATH_ARGS:
    proj_path = Path('C:\data\MattM_Ensemble')  # [CHANGE THIS for new environment]
    model_path = proj_path.parent/'model_chkpts'
    # data path
    #data_path = proj_path.parent/'TestingData'   # Test path
    data_path = proj_path.parent      # [CONFIRM THIS for new environment]
    dataindex_path = data_path/'dataIndex.csv'
    print(f"Project folder:{proj_path}, {data_path}")
    # 2 types of images (HE  FISH)
    data_name = ['HE images']
    # 2 groups to classify
    class_names = ['Responder','NonResponder']

def mkdirifNE(p):
    if not os.path.exists(p): os.mkdir(p)

mkdirifNE(PATH_ARGS.model_path)

def load_img(img_paths: list, is_mask=False):
        """ load array from a list of image paths """
        if is_mask: flag = 0
        else: flag = -1
        return np.concatenate([np.expand_dims(cv2.imread(str(img_fp), flag), axis=0)
                               for img_fp in img_paths.tolist()])
def normalize(ratios):
    """normalize a list of ratios to sum to 1"""
    return [r/sum(ratios) for r in ratios]

Project folder:C:\data\MattM_Ensemble, C:\data


In [ ]:
data_index_df = pd.read_csv(PATH_ARGS.data_path/'dataIndex.csv', index_col=0)
print(data_index_df['set'].unique())
print(data_index_df['set'].value_counts())
data_index_df.head(2)

  exec(code_obj, self.user_global_ns, self.user_ns)



['train' 'val' 'test' nan]
train    1309
test      375
val       187
Name: set, dtype: int64


,type,x_img_path,label,set
path,,,,
drive/MyDrive/Colab Notebooks/Work stuff/Acuity Diagnostic/TestingData/HIPT_AGH_FluorescentImage_R1_processed/Responder/Case26-R1-fov3-dapi-20a-140-142-40x007_t0.tif,HIPT_AGH_FluorescentImage_R1,/content/drive/MyDrive/Colab Notebooks/Work st...,Responder,train
drive/MyDrive/Colab Notebooks/Work stuff/Acuity Diagnostic/TestingData/HIPT_AGH_FluorescentImage_R1_processed/Responder/Case26-R1-fov3-dapi-20a-140-142-40x007_t1.tif,HIPT_AGH_FluorescentImage_R1,/content/drive/MyDrive/Colab Notebooks/Work st...,Responder,train


### Dataloader - loader

In [ ]:
class META_ARGS:
    rcParams['figure.figsize'] = 12, 8
    RANDOM_SEED = 42
    INPUT_DIM = (224,224)
    np.random.seed(RANDOM_SEED)
    torch.manual_seed(RANDOM_SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DATA_ARGS:
    num_classes = 2
    batch_size = 8

In [ ]:
def _get_normalize_attributes(data_index_df):
    x_imgs = load_img(data_index_df['x_img_path'])
    means, stds = np.mean(x_imgs, axis=((0,1,2))), np.std(x_imgs, axis=((0,1,2)))
    return means, stds

# dataloader
class HEData(Dataset):
    def __init__(self, dataindex_df: pd.DataFrame,
                 x_img_cols:str=['x_img_path'], y_cols:list=['label'],
                 transform=None, target_transform=None, patch_size=None, debug:bool=False):
        """ 
        parameters
            csv_file: contain indexer file
            
        """
        self.debug = debug
        # 
        self.n = len(dataindex_df)
        # fetch individual 
        self.y_ds = dataindex_df[y_cols]
        self.num_classes = self.y_ds.nunique()
        self.y_ds_enc = self.label_encode(self.y_ds, oh=False)
        # 
        self.transform = transform
        self.target_transform = target_transform
        if self.debug:
            print(f"Target shape:{self.y_ds_enc.shape}")
            print(f"[INFO]Image classes: {self.num_classes} with {self.n} instances.")
        self.patch_size = patch_size
        # if not patch_size is None:
        #     assert patch_size[0]>=224 and patch_size[0]%224==0
        
    def __len__(self):
        return self.n

    def label_encode(self, ys, oh:bool=False):
        # encode target label
        if oh:
            self.enc = OneHotEncoder()
            return self.enc.fit_transform(ys).toarray()
        else:
            self.enc = LabelBinarizer()
        ys_enc = self.enc.fit_transform(ys)
        return ys_enc.flatten()

    def __getitem__(self, idx):
        # input images
        if self.debug: print(f"Instance series: {self.y_ds.iloc[idx]},{self.y_ds.iloc[idx].name}, {idx}")
        parent_path, _, _, tile_name = self.y_ds.iloc[idx].name   # parent, type, source_tissue, tile_name
            # get data
        if not self.patch_size is None:
            x_data = np.concatenate([x for x in tile_images_basic(np.array(Image.open(Path(parent_path)/tile_name)),
                                                                  patch_dims=self.patch_size)])
        else:
            x_data = np.array(Image.open(Path(parent_path)/tile_name))
        y_data = self.y_ds_enc[idx]    #.reshape((-1,))
        if self.transform is not None:
            x_data = [self.transform(image=x)['image'] for x in x_data]
        if self.target_transform:
            y_data = self.target_transform(y_data)
        # outputs g(t)
        if self.debug: print(x_data.shape, x_data.dtype, y_data.shape, y_data.dtype)
        return (torch.tensor(x_data).float(), torch.tensor(y_data, dtype=torch.long).tile(len(x_data)))

def get_transforms(target_size=(224,224), get_normalizing_attributes:bool=False, data_index_df:pd.DataFrame=False):    
    assert bool(get_normalizing_attributes) == bool(data_index_df), "must be provided together"
    p1 = 0.1
    p2 = 0.05
    p3 = 0.2

    if get_normalizing_attributes:
        im_means, im_stds = _get_normalize_attributes()
    else:   # use pre-computed values
        im_means, im_stds=[0, 0, 0], [1, 1, 1]
    ## Transforms
    process_transform = A.Compose([
        ToTensorV2(),
    ]) # Normalize by channel means, stds
    color_transform = A.Compose([
        # In-place transformations
        A.RandomBrightnessContrast(p=p2),
        A.RandomGamma(gamma_limit=(80, 200), p=p3),
        A.Blur(blur_limit=7, p=p2),
        A.ToGray(p=p2),
        A.CLAHE(p=p2),
        A.ChannelDropout(channel_drop_range=(1, 2), fill_value=0, p=p2),
        A.ChannelShuffle(p=p2),
        A.ColorJitter(
            brightness=0.2,
            contrast=0.2,
            saturation=0.2,
            hue=0.2,
            always_apply=False,
            p=p2,
        ),
        A.Equalize(mode="cv", by_channels=True, mask=None, mask_params=(), p=p2),
        A.GaussNoise(var_limit=(10.0, 50.0), mean=0, per_channel=True, p=p2),
        A.Posterize(num_bits=4, p=p2),
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=p2),
        A.GaussianBlur(blur_limit=(3, 7), p=p1)
        #A.GaussianBlur(11, sigma=(0.1, 2.0)),
    ])
    geometric_transform = A.Compose([
        A.Affine(
            scale=(0.60, 1.60),
            interpolation=cv2.INTER_LINEAR,
            cval=0,
            cval_mask=0,
            mode=cv2.BORDER_CONSTANT,
            fit_output=False,
            p=p1,
        ),
        A.Affine(
            translate_percent=(-0.2, 0.2),
            interpolation=cv2.INTER_LINEAR,
            cval=0,
            cval_mask=0,
            mode=cv2.BORDER_CONSTANT,
            fit_output=False,
            p=p1,
        ),
        A.Affine(
            rotate=(-30, 30),
            interpolation=cv2.INTER_LINEAR,
            cval=0,
            cval_mask=0,
            mode=cv2.BORDER_CONSTANT,
            fit_output=False,
            p=p1,
        ),
        A.Affine(
            shear=(-20, 20),
            interpolation=cv2.INTER_LINEAR,
            cval=0,
            cval_mask=0,
            mode=cv2.BORDER_CONSTANT,
            fit_output=False,
            p=p1,
        ),
        # A.OpticalDistortion(distort_limit=2, shift_limit=0.5, p=pt),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
    ])
    ###
    transformers = {'process': process_transform,  
                    'color': color_transform, 'geometric': geometric_transform}
    set_transformers = {'train': A.Compose(color_transform.transforms + geometric_transform.transforms+process_transform.transforms),
                        'val': A.Compose(process_transform.transforms),
                        'test': A.Compose(process_transform.transforms)}
    return set_transformers
    #return transformers
    
#HEData(dataindex_df=pd.read_csv(PATH_ARGS.dataindex_path,index_col=list(range(4))))[0]

NameError: name 'Dataset' is not defined

In [ ]:
# full dataset objecct
class HEDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64, dataindex_path=Path('./dataIndex.csv'), label_col='label', debug=False):
        super().__init__()
        self.dataindex_path = Path(dataindex_path)
        self.batch_size = batch_size
        self.num_classes = DATA_ARGS.num_classes
        self.label_col = label_col
        self.transforms = get_transforms()
        self.debug = debug
        print(f"Debug mode:{self.debug}")
        self.index_col_len = 4
    
    def get_sampler(self, dataset):
        """get sampler if needed"""
        if self.label_col:
            class_cts = dataset[self.label_col].value_counts()
            for label in class_cts.index:
                class_cts.loc[label] = len(dataset)/class_cts.loc[label]
            weights = np.zeros(len(dataset))
            for label in class_cts.index:
                weights[np.where(dataset[self.label_col].to_numpy()==label)[0]] = class_cts.loc[label]
            class_balance_sampler = WeightedRandomSampler(weights, len(dataset), replacement=True)
        else:
            class_balance_sampler = None
        return class_balance_sampler
        
    def setup(self, stage=None):
        self.datasets = dict()
        self.sampler = dict()
        # ['train', 'test', 'val']
        dataindex_df = pd.read_csv(self.dataindex_path, index_col=list(range(self.index_col_len)))
        dataindex_df = dataindex_df[dataindex_df['set'].isnull()!=True]
        for dset in dataindex_df['set'].unique():
            self.sampler[dset] = self.get_sampler(dataindex_df[dataindex_df['set']==dset])
            self.datasets[dset] = HEData(dataindex_df[dataindex_df['set']==dset],
                                         transform = self.transforms[dset], debug=self.debug)

    def train_dataloader(self):
        train_loader = DataLoader(
            self.datasets['train'], batch_size=self.batch_size, shuffle=False if self.sampler['train'] else True, sampler=self.sampler['train'],
            num_workers=64, pin_memory=True
        )
        return train_loader

    def val_dataloader(self):
        valid_loader = DataLoader(
            self.datasets['val'], batch_size=self.batch_size, shuffle=False if self.sampler['val'] else True, sampler=self.sampler['val'],
            num_workers=64, pin_memory=
        )
        return valid_loader

### Model 

In [ ]:
# model and train args
class MODEL_ARGS:
    n_classes = len(PATH_ARGS.class_names)
    
class TRAIN_ARGS:
    batch_size = DATA_ARGS.batch_size
    epochs = 100

In [ ]:
# returns the size of the output tensor going into Linear layer from the conv block.
def _get_conv_output(self, shape):
    batch_size = 1
    input = torch.autograd.Variable(torch.rand(batch_size, *shape))

    output_feat = self._forward_features(input) 
    n_size = output_feat.data.view(batch_size, -1).size(1)
    return n_size
    
#from torch._C import device
class HEClassificationModel(pl.LightningModule):
    def __init__(self, model_name:str, n_classes:int=2, pretrain:bool=True,
                 custom_classification_head:bool=False, input_size:tuple=(224,224), debug:bool=False):
        super().__init__()
        print(f"Using pre-trained head:{model_name}")
        avail_models =  ['mobilenetv3','resnext101','efficientnetb7','resnet50']
        assert model_name in ['mobilenetv3','resnext101','efficientnetb7','resnet50'], f"Must be one of {avail_models}"
        self.debug = debug
        self.n_classes = n_classes
        self.custom_classification_head = custom_classification_head
        # Step 1: Initialize model with the weights
        if model_name == 'mobilenetv3':
            self.model = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.IMAGENET1K_V2 if pretrain else None)
        elif model_name == 'resnext101':
            self.model = models.resnext101_32x8d(weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1 if pretrain else None)
        elif model_name == 'efficientnetb7':
            self.model = models.efficientnet_b7(weights=EfficientNet_B7_Weights.IMAGENET1K_V1 if pretrain else None)
        elif model_name =='resnet50':
            self.model = models.resnet50(pretrained=ResNet50_Weights.IMAGENET1K_V2 if pretrain else None)
        # replace/remove head
        removed = list(self.model.children())[:-1]
        self.model_base = torch.nn.Sequential(*removed)  
        in_feats = self._get_output_feat(self.model_base, input_size)
            # head
        if self.custom_classification_head:
            self.model_head = self.classification_head()
        else:
            self.model_head = nn.Sequential(nn.Flatten(),
                                            nn.Linear(in_features=in_feats, out_features=self.n_classes, bias=True),
                                            nn.ReLU(),
                                            nn.LogSoftmax(dim=1) if n_classes>2 else nn.Sigmoid(),
                                           )
        self.model = torch.nn.Sequential(self.model_base, self.model_head)
            #self.model_head.to(device=META_ARGS.device)     
        # metrics
        self.accuracy = torchmetrics.Accuracy()
        #self.ROC = torchmetrics.ROC(num_classes=n_classes)
        self.AUROC = torchmetrics.AUROC(num_classes=n_classes, pos_label=1)
    
    def _get_output_feat(self, model, in_shape=(224,224)):
        x = torch.randn((3,)+in_shape)
        return model(x.unsqueeze(0)).flatten().size()[0]

    def _forward_feature_extract(self, x):
        return self.model_base(x)

    def forward(self, x):
        x = self.model(x)
        
#         #x = self.model_head(x)
#         x = x.view(x.size(0), -1)
#         x = F.relu(nn.Linear(in_features=self.model.classifier[-1].in_features, out_features=self.n_classes, bias=True)(x))
#         x = F.log_softmax(x, dim=1)
        #self.model.classifier = nn.Sequential(*self.model.classifier, nn.Softmax())
        if self.debug: print(f"Num classes:{self.n_classes}\nModel classifier\n:{self.model_head}")
        return x

    def add_classification_head(self):
        #n_features = self.model_head.fc.in_features
        classifier_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.model_base.classifier[1].in_features, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            nn.Linear(512 , 256),
            nn.Linear(256 , self.n_classes),
            nn.Softmax(dim=1)
            )
        return classifier_layer

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-10)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [lr_scheduler]

    def get_loss(self, y_hat, y):
        #loss = nn.CrossEntropyLoss()   # does softmax for you (no need in classifcation)
        #loss = nn.LogSoftmax()
        #loss = F.nll_loss
        if self.debug: print(y.size(), y.dtype, y_hat.size(), y_hat.dtype)
        return F.cross_entropy(y_hat,  y)

    def training_step(self, batch, batch_idx=None):
        x, y = batch
        y_hat = self(x)
        loss = self.get_loss(y_hat, y)
        # training metrics
        acc = self.accuracy(torch.argmax(y_hat, dim=1), y)
        # optimize (done under the hoood)

        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
        return loss
        #return self.get_loss(y, y_hat)

    def validation_step(self, batch, batch_idx=None):
        x, y = batch
        y_hat = self(x)
        # compute metrics
        val_loss =self.get_loss(y_hat, y)
        acc = self.accuracy(torch.argmax(y_hat, dim=1), y)
        auroc = self.AUROC(y_hat.cpu().detach(), y.cpu().detach())
        #fpr, tpr, thresholds = self.ROC(y_hat, y)
        #
        self.log("val_loss", val_loss)
        self.log('val_acc', acc, on_step=True, on_epoch=True, logger=True)
        self.AUROC.update(y_hat.cpu().detach(), y.cpu().detach())
        self.log("validation_auc", self.AUROC, on_step=False, on_epoch=True, prog_bar=True)
        #self.log("val_auc", valid_auc, on_step=False, on_epoch=True, prog_bar=True)
        

class HEEnsembleModel(pl.LightningModule):
    def __init__(self, 
                 ensembles_settings:dict={'efficientnetb7':3, 'resnext101':2}, 
                 pretrain:bool=True,
                 n_classes:int=2,
                 input_shape=(224,224),
                 debug=False):
        super(HEEnsembleModel, self).__init__()
        self.debug = debug
        models = []
        self.n_models = 0
        for name, number in ensembles_settings.items():
            [models.append(
                HEClassificationModel(model_name=name, 
                                      n_classes=2, 
                                      pretrain=pretrain,
                                      custom_classification_head=False
                                     )
                         ) for i in range(number)
            ]
            self.n_models += number
        self.ensemble_model = torch.nn.ModuleList(models)
        self.classifier = torch.nn.Linear(self.n_models*n_classes, n_classes)
        #self.save_hyperparameters() # Uncomment to show error
        self.CEloss = nn.CrossEntropyLoss()
        # metrics
        self.accuracy = torchmetrics.Accuracy()
        #self.ROC = torchmetrics.ROC(num_classes=n_classes)
        self.AUROC = torchmetrics.AUROC(num_classes=n_classes, pos_label=1)
        
    def forward(self, x):
        output=[]
        for m in self.ensemble_model:
            output.append(m(x))
        combined = torch.concat(output,dim=1)
        x = self.classifier(combined)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4, weight_decay=1e-10)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [lr_scheduler]

    def get_loss(self, y_hat, y):
        #loss = nn.CrossEntropyLoss()   # does softmax for you (no need in classifcation)
        #loss = nn.LogSoftmax()
        #loss = F.nll_loss
        if self.debug: print(y.size(), y.dtype, y_hat.size(), y_hat.dtype)
        return self.CEloss(y_hat,  y)

    def training_step(self, batch, batch_idx=None):
        x, y = batch
        y_hat = self(x)
        loss = self.get_loss(y_hat, y)
        # training metrics
        acc = self.accuracy(torch.argmax(y_hat, dim=1), y)
        # optimize (done under the hoood)

        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
        return loss
        #return self.get_loss(y, y_hat)

    def validation_step(self, batch, batch_idx=None):
        x, y = batch
        y_hat = self(x)
        # compute metrics
        val_loss =self.get_loss(y_hat, y)
        acc = self.accuracy(torch.argmax(y_hat, dim=1), y)
        auroc = self.AUROC(y_hat.cpu().detach(), y.cpu().detach())
        #fpr, tpr, thresholds = self.ROC(y_hat, y)
        #
        self.log("val_loss", val_loss)
        self.log('val_acc', acc, on_step=True, on_epoch=True, logger=True)
        self.AUROC.update(y_hat.cpu().detach(), y.cpu().detach())
        self.log("validation_auc", self.AUROC, on_step=False, on_epoch=True, prog_bar=True)
        #self.log("val_auc", valid_auc, on_step=False, on_epoch=True, prog_bar=True)
    

In [ ]:
    # callbacks
class PRMetrics(pl.Callback):
    """ Custom callback to compute per-class PR & ROC curves
    at the end of each training epoch"""
    def __init__(self,  val_samples, num_samples=32, class_names={'Non-responder':0, 'Responder':1}):    #generator=None, num_log_batches=1):
        # self.generator = generator
        # self.num_batches = num_log_batches
        # # store full names of classes
        # self.class_names = { v: k for k, v in generator.class_indices.items() }
        # self.flat_class_names = [k for k, v in generator.class_indices.items()]

        super().__init__()
        self.num_samples = num_samples
        self.class_names = class_names
        self.val_imgs, self.val_labels = val_samples

    def on_epoch_end(self, trainer, pl_module, logs={}):
        # # collect validation data and ground truth labels from generator
        # val_data, val_labels = zip(*(self.generator[i] for i in range(self.num_batches)))
        # val_data, val_labels = np.vstack(val_data), np.vstack(val_labels)

        # # use the trained model to generate predictions for the given number
        # # of validation data batches (num_batches)
        # val_predictions = self.model.predict(val_data)
        # ground_truth_class_ids = val_labels.argmax(axis=1)
        # Bring the tensors to CPU
        val_imgs = self.val_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        # Get model prediction
        preds = torch.argmax(pl_module(val_imgs), -1)
        # Log precision-recall curve the key "pr_curve" is the id of the plot--do not change this if you want subsequent runs to show up on the same plot
        wandb.log({"roc_curve" : wandb.plot.roc_curve(val_labels, preds, labels=self.class_names)})

class ImagePredictionLogger(pl.Callback):
    """ callback"""
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.num_samples = num_samples
        self.val_imgs, self.val_labels = val_samples

    def on_validation_epoch_end(self, trainer, pl_module):
        # Bring the tensors to CPU
        val_imgs = self.val_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        # Get model prediction
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, -1)
        # Log the images as wandb Image
        trainer.logger.experiment.log({
            "examples":[wandb.Image(x, caption=f"Pred:{pred}, Label:{y}") 
                           for x, pred, y in zip(val_imgs[:self.num_samples], 
                                                 preds[:self.num_samples], 
                                                 val_labels[:self.num_samples])]
        })

### Training

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold
from pytorch_lightning.callbacks import EarlyStopping, GradientAccumulationScheduler
import pytorch_lightning as pl
# logger
from pytorch_lightning.loggers import WandbLogger
#
from torch.optim import lr_scheduler
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
class TRAIN_args:
    batch_size = 4
    epochs = 100

In [ ]:
def reset_weights(m):
    '''
        Try resetting model weights to avoid
        weight leakage.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [ ]:
# DEFAULT (ie: no accumulated grads)
cbs = [
    pl.callbacks.ModelCheckpoint(monitor='val_loss', dirpath=PATH_ARGS.model_path,
                                 filename='models-{epoch:02d}-{val_loss:.2f}', save_top_k=2, mode='min'),
    EarlyStopping(monitor="val_loss", min_delta=1e-7, patience=8, mode="min"),
    GradientAccumulationScheduler(scheduling={0: 8, 4: 4, 8: 1}),
    
]
trainer = pl.Trainer(
    #gpus=1, 
    devices=1,
    logger=WandbLogger(project='AD-ensemble(draft)',  entity="3m-m", job_type='train'),
    max_epochs=TRAIN_args.epochs, callbacks=cbs)

model = HEClassificationModel(model_name='efficientnetb7',
                              n_classes=MODEL_ARGS.n_classes)
datamodule = HEDataModule(batch_size=DATA_ARGS.batch_size)
datamodule.setup()

TypeError: ignored

In [ ]:
# train
trainer.fit(model=model, datamodule=datamodule) 
# save with parameters
#torch.save([model.kwargs, model.state_dict()], path)

  # Remove the CWD from sys.path while we load stuff.

  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type         | Params
--------------------------------------------
0 | model      | EfficientNet | 66.3 M
1 | model_base | Sequential   | 63.8 M
2 | model_head | Sequential   | 5.1 K 
3 | accuracy   | Accuracy     | 0     
4 | AUROC      | AUROC        | 0     
--------------------------------------------
66.4 M    Trainable params
0         Non-trainable params
66.4 M    Total params
265.412   Total estimated model params size (MB)


<class 'str'> train
Target shape:(1309,)
[INFO]Image classes: 2 with 1309 instances.
<class 'str'> val
Target shape:(187,)
[INFO]Image classes: 2 with 187 instances.
<class 'str'> test
Target shape:(375,)
[INFO]Image classes: 2 with 375 instances.


Sanity Checking: 0it [00:00, ?it/s]

  warnings.warn(*args, **kwargs)

  warnings.warn(*args, **kwargs)

  warnings.warn(*args, **kwargs)



Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

### Evaluate -ongoing

In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass

def best_model_from_fp(model_path=PATH_ARGS.model_path):
    fs = [Path(s).stem.split('val_loss=')[-1] for s in os.listdir(model_path) if s.startswith('models')]
    max_idx = np.argmax(list(map(float, fs)))
    return os.listdir(model_path)[max_idx]

model_chkpt = best_model_from_fp(PATH_ARGS.model_path)
print(PATH_ARGS.model_path/model_chkpt)


/content/drive/MyDrive/Colab Notebooks/Work stuff/Acuity Diagnostic/model_chkpts/models-epoch=01-val_loss=0.70.ckpt


In [ ]:
model = HEEnsembleModel(ensembles_settings={'efficientnetb7':1,
                                            'mobilenetv3':2,
                                            'resnext101':2},
                        pretrain=False,
                        input_shape=(224,224),
                        n_classes=MODEL_ARGS.n_classes,
                        debug=False)

Using pre-trained head:efficientnetb7
Using pre-trained head:efficientnetb7


In [ ]:
class BaseKFoldDataModule(LightningDataModule, ABC):
    @abstractmethod
    def setup_folds(self, num_folds: int) -> None:
        pass

    @abstractmethod
    def setup_fold_index(self, fold_index: int) -> None:
        pass
        
@dataclass
class HEKFoldDataModule(BaseKFoldDataModule):
    # train_dataset: Optional[Dataset] = None
    # test_dataset: Optional[Dataset] = None
    # train_fold: Optional[Dataset] = None
    # val_fold: Optional[Dataset] = None

    def prepare_data(self, dataindex_path, batchsize) -> None:
        # download the data.
        MNIST(DATASETS_PATH, transform=T.Compose([T.ToTensor(), T.Normalize(mean=(0.5,), std=(0.5,))]))
        self.datamodule=HEDataModule(batch_size=batchsize, dataindex_path=dataindex_path, debug=False)


    def setup(self, stage: Optional[str] = None) -> None:
        # load the data
        dataset = MNIST(DATASETS_PATH, transform=T.Compose([T.ToTensor(), T.Normalize(mean=(0.5,), std=(0.5,))]))
        self.train_dataset, self.test_dataset = random_split(dataset, [50000, 10000])

    def setup_folds(self, num_folds: int) -> None:
        self.num_folds = num_folds
        self.splits = [split for split in KFold(num_folds).split(range(len(self.train_dataset)))]

    def setup_fold_index(self, fold_index: int) -> None:
        train_indices, val_indices = self.splits[fold_index]
        self.train_fold = Subset(self.train_dataset, train_indices)
        self.val_fold = Subset(self.train_dataset, val_indices)

    def train_dataloader(self) -> DataLoader:
        return DataLoader(self.train_fold)

    def val_dataloader(self) -> DataLoader:
        return DataLoader(self.val_fold)

    def test_dataloader(self) -> DataLoader:
        return DataLoader(self.test_dataset)

    def __post_init__(cls):
        super().__init__()


In [ ]:
class KFoldLoop(Loop):
    def __init__(self, num_folds: int, export_path: str) -> None:
        super().__init__()
        self.num_folds = num_folds
        self.current_fold: int = 0
        self.export_path = export_path

    @property
    def done(self) -> bool:
        return self.current_fold >= self.num_folds

    def connect(self, fit_loop: FitLoop) -> None:
        self.fit_loop = fit_loop

    def reset(self) -> None:
        """Nothing to reset in this loop."""

    def on_run_start(self, *args: Any, **kwargs: Any) -> None:
        """Used to call `setup_folds` from the `BaseKFoldDataModule` instance and store the original weights of the
        model."""
        assert isinstance(self.trainer.datamodule, BaseKFoldDataModule)
        self.trainer.datamodule.setup_folds(self.num_folds)
        self.lightning_module_state_dict = deepcopy(self.trainer.lightning_module.state_dict())

    def on_advance_start(self, *args: Any, **kwargs: Any) -> None:
        """Used to call `setup_fold_index` from the `BaseKFoldDataModule` instance."""
        print(f"STARTING FOLD {self.current_fold}")
        assert isinstance(self.trainer.datamodule, BaseKFoldDataModule)
        self.trainer.datamodule.setup_fold_index(self.current_fold)

    def advance(self, *args: Any, **kwargs: Any) -> None:
        """Used to the run a fitting and testing on the current hold."""
        self._reset_fitting()  # requires to reset the tracking stage.
        self.fit_loop.run()

        self._reset_testing()  # requires to reset the tracking stage.
        self.trainer.test_loop.run()
        self.current_fold += 1  # increment fold tracking number.

    def on_advance_end(self) -> None:
        """Used to save the weights of the current fold and reset the LightningModule and its optimizers."""
        self.trainer.save_checkpoint(osp.join(self.export_path, f"model.{self.current_fold}.pt"))
        # restore the original weights + optimizers and schedulers.
        self.trainer.lightning_module.load_state_dict(self.lightning_module_state_dict)
        self.trainer.strategy.setup_optimizers(self.trainer)
        self.replace(fit_loop=FitLoop)

    def on_run_end(self) -> None:
        """Used to compute the performance of the ensemble model on the test set."""
        checkpoint_paths = [osp.join(self.export_path, f"model.{f_idx + 1}.pt") for f_idx in range(self.num_folds)]
        voting_model = EnsembleVotingModel(type(self.trainer.lightning_module), checkpoint_paths)
        voting_model.trainer = self.trainer
        # This requires to connect the new model and move it the right device.
        self.trainer.strategy.connect(voting_model)
        self.trainer.strategy.model_to_device()
        self.trainer.test_loop.run()

    def on_save_checkpoint(self) -> Dict[str, int]:
        return {"current_fold": self.current_fold}

    def on_load_checkpoint(self, state_dict: Dict) -> None:
        self.current_fold = state_dict["current_fold"]

    def _reset_fitting(self) -> None:
        self.trainer.reset_train_dataloader()
        self.trainer.reset_val_dataloader()
        self.trainer.state.fn = TrainerFn.FITTING
        self.trainer.training = True

    def _reset_testing(self) -> None:
        self.trainer.reset_test_dataloader()
        self.trainer.state.fn = TrainerFn.TESTING
        self.trainer.testing = True

    def __getattr__(self, key) -> Any:
        # requires to be overridden as attributes of the wrapped loop are being accessed.
        if key not in self.__dict__:
            return getattr(self.fit_loop, key)
        return self.__dict__[key]

    def __setstate__(self, state: Dict[str, Any]) -> None:
        self.__dict__.update(state)

In [ ]:
for fold, (train_idx, test_idx) in enumerate(KFold.split(HEdatasets)):
    print('------------fold no---------{}----------------------'.format(fold))
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)
    # Train 
    train_loader = DataLoader(HEdatasets['train'], shuffle=True,
                             batch_size=TRAIN_ARGS.batch_size, sampler=train_subsampler)
    

    model.apply(reset_weights)

    for epoch in range(1, TRAIN_ARGS.MAX_EPOCHS + 1):
        train(fold, model, device, trainloader, optimizer, epoch)
        test(fold,model, device, testloader)
    test_loader = DataLoader(HEdatasets['test'], shuffle=True,
                            batch_size=TRAIN_ARGS.batch_size, sampler=test_subsampler)

In [ ]:
def train_epoch(
    model,
    data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    n_examples
    ):
    model = model.train()
    losses = []
    correct_predictions = 0
    for inputs, labels in data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()
    return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def train_model(model, dataloaders, dataset_size, device, n_epochs):
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    loss_fn = nn.CrossEntropyLoss().to(device)
    #
    history = defaultdict(list)
    best_accuracy = 0
    # loop training
    for epoch in range(n_epochs):
        train_acc, train_loss = train_epoch(
            model, data_loaders['train'], loss_fn,
            optimizer, device, scheduler, dataset_sizes['train'])
        history['train_acc'].append(train_acc)
        history['train_loss'].append(train_loss)
        history['val_acc'].append(val_acc)
        history['val_loss'].append(val_loss)

        if val_acc > best_accuracy:
            torch.save(model.state_dict(), 'best_model_state.bin')
        best_accuracy = val_acc
        print(f'Best val accuracy: {best_accuracy}')
        model.load_state_dict(torch.load('best_model_state.bin'))
        return model, history

### Prediction/submission

In [ ]:
for data in test_loader:
  y_hat = pretrained_model(data["x"].view(-1,3,IMG_SIZE,IMG_SIZE))
  y_hat = nn.Softmax(dim=-1)(y_hat)
  y_hat = torch.argmax(y_hat,dim=1)
  fin_y.append(y_hat.cpu().detach().numpy())
sample["label"] = np.array(fin_y).reshape(-1)
sample[["image_id","label"]].to_csv("submission.csv",index=False)
sample.head()